# [Module 4.2.2] inference on custem tf serving image

이 노트북은 아래와 같은 작업을 진행 합니다.
- Custom Inference Docker Image를 사용하여, SageMaker Model을 생성
- 위의 SageMaker Model를 통해서 Deploy 하여 Endpoint를 생성 합니다.
- Endpoint에 Predictor를 생성 합니다.
- 추론을 하여 상위 5개의 높은 스코어를 가진 것을 이모팀콘으로 변경하여 5개 추천 합니다.

---
이 노트북은 약 10 분 정도 시간이 소요 됩니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
%store -r 

In [3]:
# training_job_name = 'bert2tweet-2020-07-08-07-58-27-895'
print(training_job_name)

bert2tweet-2020-08-17-10-11-25-431


In [4]:
inference_image = ecr_infer_custom_image_tf_serving_20_cpu
print("inference_image: ", inference_image)

inference_image:  343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


## Custome TFS Docker Image 및 Inference code 로 모델 생성

In [5]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

In [6]:
print(model.image)
print(model.serving_image_uri)
print(model.model_data)
print(model.name)
print(model.FRAMEWORK_NAME)


343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu
<bound method Model.serving_image_uri of <sagemaker.tensorflow.serving.Model object at 0x7fcbf1560588>>
s3://sagemaker-ap-northeast-2-343441690612/bert2tweet-2020-08-17-10-11-25-431/output/model.tar.gz
None
tensorflow-serving


## 엔드포인트 생성

In [7]:
%%time

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


-------------!CPU times: user 1min 4s, sys: 11 s, total: 1min 15s
Wall time: 7min 41s


## Predictor Creation on the Endpoint

In [8]:
# tweet_bert_endpoint_name = 'train_text, train_label, test_text, test_label = tweet_data.split_train_test_data(texts, labels, 0.9)
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-08-17-10-30-49-226


In [9]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [10]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [11]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
2004,bitches be hating on me and haven't even met me,6
5004,i'm so lost without you,6
7144,in case anyone needs this i love this,0
3141,gucci said he wanna work with 21 savage im re...,2
2862,rt yall lettin niggas who shoot like this beat,7
3260,had your first kiss sadly no,6
7670,purpose tour,2
2906,the lady at the hair cuttery just called me he...,8
3396,super proud of tonight witchoooo grown asss,2
7485,i'm very normal i'm an approachable standoffi...,8


In [12]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


## Top 5 이모티콘 추천

In [16]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 5
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]
    
    print("tweet: \n", reviews)



    predicted_classes = predictor.predict(reviews)[0]
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    

    print('Prediction: {},{},{},{},{},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2]),   
        predicted_classes[3], 
        tweet_util.get_emo_class_label(predicted_classes[3]),                                      
        predicted_classes[4], 
        tweet_util.get_emo_class_label(predicted_classes[4]),                                      
        
        ))    

        



tweet: 
 [" bitches be hating on me and haven't even met me"]
tweet:  bitches be hating on me and haven't even met me 
Ground_truth- 6:😩
 
Prediction: 6,😩,8,🙄,7,😭,5,😍,1,💕 
 
tweet: 
 [" i'm so lost without you"]
tweet:  i'm so lost without you 
Ground_truth- 6:😩
 
Prediction: 6,😩,1,💕,7,😭,5,😍,9,🤔 
 
tweet: 
 [' in case anyone needs this i love this']
tweet:  in case anyone needs this i love this 
Ground_truth- 0:❤
 
Prediction: 5,😍,6,😩,1,💕,7,😭,0,❤ 
 
tweet: 
 [' gucci said he wanna work with 21 savage im ready for that']
tweet:  gucci said he wanna work with 21 savage im ready for that 
Ground_truth- 2:🔥
 
Prediction: 1,💕,2,🔥,0,❤,7,😭,9,🤔 
 
tweet: 
 ['rt yall lettin niggas who shoot like this beat ']
tweet: rt yall lettin niggas who shoot like this beat  
Ground_truth- 7:😭
 
Prediction: 7,😭,3,😂,5,😍,1,💕,2,🔥 
 
tweet: 
 ['had your first kiss sadly no']
tweet: had your first kiss sadly no 
Ground_truth- 6:😩
 
Prediction: 5,😍,6,😩,0,❤,1,💕,8,🙄 
 
tweet: 
 [' purpose tour']
tweet:  purpose tou

In [14]:
%store tweet_bert_endpoint_name

Stored 'tweet_bert_endpoint_name' (str)
